# French Departmental/Regional Data

In [204]:
import pandas as pd

----
&nbsp;
## Creating new columns `department` & `region`

Compared to the UK France is considerably more ordered in terms of how she defines regions.

1. We extract the first two digits of the code postal from `address`. ie, 06, 78 etc
2. These digits represent a particular department
3. We arrogate these into regions

We search for the departments and regions of france on [Wikipedia](https://en.wikipedia.org/wiki/Departments_of_France) and scrape the data using pandas

In [205]:
url = 'https://en.wikipedia.org/wiki/Departments_of_France'
tables = pd.read_html(url)

In [206]:
# The table of interest is found by inspection
departments = tables[4]
departments.head()

,INSEE code,Arms 1,Date of establishment,Department,Capital,Region,Named after
0,01,NaN,26 February 1790,Ain,Bourg-en-Bresse,Auvergne-Rhône-Alpes,Ain (river)
1,02,NaN,26 February 1790,Aisne,Laon,Hauts-de-France,Aisne (river)
2,03,NaN,26 February 1790,Allier,Moulins,Auvergne-Rhône-Alpes,Allier (river)
3,04,NaN,26 February 1790,Alpes-de-Haute-Provence 2,Digne-les-Bains,Provence-Alpes-Côte d'Azur,Alps mountains and Provence region
4,05,NaN,26 February 1790,Hautes-Alpes,Gap,Provence-Alpes-Côte d'Azur,Alps mountains


In [207]:
print(departments.columns.tolist())

['INSEE code', 'Arms 1', 'Date of establishment', 'Department', 'Capital', 'Region', 'Named after']


In [208]:
# Drop specified columns from the DataFrame
departments = departments.drop(columns=['Arms 1', 'Date of establishment', 'Capital', 'Named after'])

# Rename the column 'INSEE code' to 'department_num'
departments = departments.rename(columns={'INSEE code': 'department_num'})

# Convert all column names in the DataFrame to lowercase
departments.columns = departments.columns.str.lower()

# Replace the trailing numbers in the 'department' column with an empty string (if it exists)
departments['department'] = departments['department'].str.replace(r'\d+$', '', regex=True)

# Display the last 10 rows of the DataFrame
departments.tail(10)

,department_num,department,region
92,91,Essonne,Île-de-France
93,92,Hauts-de-Seine,Île-de-France
94,93,Seine-Saint-Denis,Île-de-France
95,94,Val-de-Marne,Île-de-France
96,95,Val-d'Oise,Île-de-France
97,971,Guadeloupe,Guadeloupe
98,972,Martinique,Martinique
99,973,Guyane,French Guiana
100,974,La Réunion,Réunion
101,976,Mayotte,Mayotte


The last five entries are overseas departments

In [209]:
# Remove the last 5 entries from the DataFrame
departments = departments.iloc[:-5]

# Add Monaco
departments.loc[len(departments)] = ['98', 'Monaco', 'Monaco']

In `departments` there is some ambiguity with the naming of department 69. We search `france_data` for department 69

In [210]:
rhone = departments[departments['department_num'].str.startswith('69')]
rhone

,department_num,department,region
69,69D,Rhône,Auvergne-Rhône-Alpes
70,69M,Lyon Metropolis,Auvergne-Rhône-Alpes


We will change department 69_ to 69 Rhône

In [211]:
# Identify rows with department number '69D' or '69M'
rows_to_update = departments['department_num'].isin(['69D', '69M'])

# Update department name and department number for these rows
departments.loc[rows_to_update, 'department'] = 'Rhône'
departments.loc[rows_to_update, 'department_num'] = '69'

# Drop the specified index
departments = departments.drop(departments.index[69])

# Reset the index
departments = departments.reset_index(drop=True)

We export `departments` as a .csv file to merge with `france_data`

In [212]:
# Export the data to a csv file
departments.to_csv('../../data/France/departments.csv', index=False)

In [213]:
departments.tail(3)

,department_num,department,region
94,94,Val-de-Marne,Île-de-France
95,95,Val-d'Oise,Île-de-France
96,98,Monaco,Monaco


In [214]:
# Removing Monaco again
departments = departments.iloc[:-1]

----
&nbsp;
## Total population data by department. [INSEE](https://www.insee.fr/fr/statistiques/6013867) 2019 Census

In [215]:
url = 'https://www.insee.fr/fr/statistiques/6013867'
tables = pd.read_html(url)

In [216]:
pop_total = tables[0]
pop_total.head()

,Code département,Nom du département,Population municipale
0,01,Ain,652 432
1,02,Aisne,531 345
2,03,Allier,335 975
3,04,Alpes-de-Haute-Provence,164 308
4,05,Hautes-Alpes,141 220


In [217]:
pop_total = pop_total.rename(columns={'Code département': 'department_num',
                                      'Nom du département': 'department',
                                      'Population municipale': 'population'})

In [218]:
# Remove whitespace from 'population' column and format as type integer
pop_total['population'] = pop_total['population'].replace(' ', '', regex=True).astype(int)

In [219]:
pop_total.tail(10)

,department_num,department,population
90,90,Territoire de Belfort,141318
91,91,Essonne,1301659
92,92,Hauts-de-Seine,1624357
93,93,Seine-Saint-Denis,1644903
94,94,Val-de-Marne,1407124
95,95,Val-d'Oise,1249674
96,971,Guadeloupe,384239
97,972,Martinique,364508
98,973,Guyane,281678
99,974,La Réunion,861210


We remove the overseas departments

In [220]:
# Remove the last 4 entries from the DataFrame
pop_total = pop_total.iloc[:-4]

----
&nbsp;
## Population density by department. [INSEE](https://statistiques-locales.insee.fr/#c=indicator&i=pop_depuis_1876.dens&s=2020&t=A01&view=map2) 2020

In [221]:
pop_density = pd.read_csv("../../data/France/france_pop_den.csv", sep=';')
pop_density.head()

,Code,Libellé,Densité de population 2020
0,01,Ain,114.2
1,02,Aisne,71.9
2,03,Allier,45.7
3,04,Alpes-de-Haute-Provence,23.9
4,05,Hautes-Alpes,25.3


In [222]:
pop_density.columns.tolist()

['Code', 'Libellé', 'Densité de population 2020']

In [223]:
pop_density.tail(10)

,Code,Libellé,Densité de population 2020
91,91,Essonne,723.9
92,92,Hauts-de-Seine,9260.4
93,93,Seine-Saint-Denis,7008.6
94,94,Val-de-Marne,5746.1
95,95,Val-d'Oise,1004.7
96,971,Guadeloupe,235.5
97,972,Martinique,320.2
98,973,Guyane,3.4
99,974,La Réunion,344.7
100,976,Mayotte,N/A - résultat non disponible


In [224]:
pop_density = pop_density.rename(columns={'Code': 'department_num',
                                      'Libellé': 'department',
                                      'Densité de population 2020': 'population_density'})

# Removing the overseas departments
pop_density = pop_density.iloc[:-5]

# Convert the density column as type 'float'
pop_density['population_density'] = pop_density['population_density'].astype(float)
pop_density.head()

,department_num,department,population_density
0,01,Ain,114.2
1,02,Aisne,71.9
2,03,Allier,45.7
3,04,Alpes-de-Haute-Provence,23.9
4,05,Hautes-Alpes,25.3


In [225]:
pop_density.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   department_num      96 non-null     object 
 1   department          96 non-null     object 
 2   population_density  96 non-null     float64
dtypes: float64(1), object(2)
memory usage: 2.4+ KB


----
&nbsp;
### Merging `departments`, `pop_total` & `pop_density`

In [226]:
# We sort all dfs in the same manner

departments = departments.sort_values('department_num')
pop_total = pop_total.sort_values('department_num')
pop_density = pop_density.sort_values('department_num')

We check if the name `department` is equal in the three dataframes

In [227]:
set1 = set(departments['department'].unique())
set2 = set(pop_total['department'].unique())
set3 = set(pop_density['department'].unique())

print(set1 == set2 == set3)  # This should print True if all sets are equal

False


In [228]:
print("In df1 but not in df2 or df3: ", set1 - set2 - set3)
print("In df2 but not in df1 or df3: ", set2 - set1 - set3)
print("In df3 but not in df1 or df2: ", set3 - set1 - set2)

In df1 but not in df2 or df3:  {'Paris ', 'Yvelines ', 'Haute-Corse ', 'Essonne ', 'Pyrénées-Atlantiques ', 'Alpes-de-Haute-Provence ', 'Gironde ', 'Hauts-de-Seine ', 'Seine-Maritime ', 'Loire-Atlantique ', 'Corse-du-Sud ', "Côtes-d'Armor ", 'Maine-et-Loire ', 'Seine-Saint-Denis ', 'Charente-Maritime '}
In df2 but not in df1 or df3:  set()
In df3 but not in df1 or df2:  set()


`departments['department']` has trailing whitespace on a few entries!

In [229]:
departments['department'] = departments['department'].str.strip()
# Redefining set1
set1 = set(departments['department'].unique())

In [230]:
print("In df1 but not in df2 or df3: ", set1 - set2 - set3)
print("In df2 but not in df1 or df3: ", set2 - set1 - set3)
print("In df3 but not in df1 or df2: ", set3 - set1 - set2)

In df1 but not in df2 or df3:  set()
In df2 but not in df1 or df3:  set()
In df3 but not in df1 or df2:  set()


In [231]:
from functools import reduce

# List of dataframes to merge
dfs = [departments, pop_total, pop_density]

# Use reduce and merge to merge all dataframes
demographics = reduce(lambda left,right: pd.merge(left,right,on=['department', 'department_num']), dfs)

In [232]:
demographics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   department_num      96 non-null     object 
 1   department          96 non-null     object 
 2   region              96 non-null     object 
 3   population          96 non-null     int64  
 4   population_density  96 non-null     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.5+ KB


By inspection, the population density of [Paris](https://en.wikipedia.org/wiki/Paris) is incorrect

In [233]:
demographics.loc[demographics['department'] == 'Paris', 'population_density'] = 20545.0

We will calculate the area of each department from `population` and `population_density`

In [234]:
demographics['area_sq_km'] = round(demographics['population'] / demographics['population_density'], 2)
demographics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   department_num      96 non-null     object 
 1   department          96 non-null     object 
 2   region              96 non-null     object 
 3   population          96 non-null     int64  
 4   population_density  96 non-null     float64
 5   area_sq_km          96 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 5.2+ KB


The `demographics` data can now be exported

In [235]:
# Export the data to a csv file
demographics.to_csv('../../data/France/demographics.csv', index=False)